In [117]:
using Evolutionary
using Statistics
using Random

In [122]:
function runge_kutta_method(f::Function, x::LinRange, initial_condition::Vector, params::Vector = [])::Tuple{LinRange, Array}
    n = length(x) - 1
    solution = Array{Float64}(undef, n+1, length(initial_condition))
    solution[1, :] =  initial_condition
    h = x[2] - x[1]  
    for i in 1:n
        k₀ = h * f(solution[i, :], params...)
        k₁ = h * f(solution[i, :] + 0.5 * k₀, params...)
        k₂ = h * f(solution[i, :] + 0.5 * k₁, params...)
        k₃ = h * f(solution[i, :] + k₂, params...)
        solution[i+1, :] = solution[i, :] + (k₀ + 2*(k₁ + k₂) + k₃) / 6
    end
    x, solution
end

runge_kutta_method (generic function with 6 methods)

In [123]:
function SIR_derivatives(SIR_numbers::Vector{Float64}, β::Number, γ::Number)::Vector{Float64}
    S, I, R = SIR_numbers
    N = sum(SIR_numbers)
    dS = -β*S*I/N  
    dI = β*S*I/N - γ*I  
    dR = γ*I  
    [dS, dI, dR]
end

SIR_derivatives (generic function with 1 method)

In [124]:
function mean_relative_error(xs::Vector{Number}, ys::Vector{Number})::Number 
    mean(abs.(xs .- ys)./xs)
end

mean_relative_error (generic function with 2 methods)

In [125]:
β = 0.2 
γ = 0.1
x, data = runge_kutta_method(SIR_derivatives, LinRange(0.0, 109.0, 1091), [235.0, 14.0, 0.0], [β, γ])
I = data[:, 2];

In [126]:
function fitness_func(solution)
    x, s = runge_kutta_method(SIR_derivatives, LinRange(0.0, 109.0, 1091), [235.0, 14.0, 0.0], solution)
    I_m = s[:, 2]
    mean_relative_error(I, I_m)
end

fitness_func (generic function with 1 method)

In [129]:
Evolutionary.optimize(
    fitness_func, 
    BoxConstraints([0.0, 0.0], [1.0, 1.0]),
    rand(2),
    GA(populationSize = 100, selection = susinv, crossover = AX, mutation = uniform(0.1), mutationRate = 0.5, ε = 0.1)
)


 * Status: success

 * Candidate solution
    Minimizer:  [0.20016070429261285, 0.10010828506733484]
    Minimum:    0.002767875880251137
    Iterations: 68

 * Found with
    Algorithm: GA[P=100,x=0.8,μ=0.5,ɛ=0.1]

 * Convergence measures
    |f(x) - f(x')| = 0.0 ≤ 1.0e-12

 * Work counters
    Seconds run:   26.682 (vs limit Inf)
    Iterations:    68
    f(x) calls:    6900
